In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***Thanks for passing by this kernel , this is Marvin Garcia, Giving another approach to this Data set , It is a bit large becasue I went through all the variables*** 
1. You will find that I divided them into Numerical (integer, float) and Categorical (ordinal,Nominal)
2. I worked train-test combined as data, and train set parallely, because I wanted to make sure about variables/SalePrice correlation
3. I tried to fill the missing values as much as I can
4. I only used Linear Regression and Xboost Because I wanted to prove that the data processing it is more important than the algorithm , then I used Xboost Randomsearch Cv for tunning for a better result 
5. I will keep working on this data set and try another algorithms... 
6. anyfeedback, comment it is more than welcome 
7. One more time thanks to all kaggle's member I have learned a lot by doing this 
8. take this notebook and if you want try to make some changes remove more variable (becaue there are more variable that can be removed and outlier)

In [ ]:
# Analyses 
import pandas as pd 
import numpy as np 
import math, re
from scipy import stats
from scipy.stats import norm, skew

#Visualization 
import seaborn as  sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from nltk.tokenize import sent_tokenize,word_tokenize
%matplotlib inline 
sns.set_style('whitegrid')


#
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None

from IPython.display import display
from PIL import Image
path=('../input/houses-2/houses.jfif')
display(Image.open(path))

> ### Special Thanks to @Mehmet Sungur, the box plot, histogram were taken from his notebook .. Awesome..!!!

# A. Let's Start With the Target Variable SalePrice

In [ ]:
#-->Importing Data
train= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")


#---> Concat iloc[:,:-1] is excluding 'SalePrice'
data  = pd.concat([train,test], axis = 0 , sort = False).reset_index(drop=True)
data.drop('SalePrice', axis =1 , inplace=True)

In [ ]:
# --- > General Fuctions

def multi_plotting (df, feature): 

    fig = plt.figure(constrained_layout=True, figsize=(12,8))
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)

    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Histogram')
    sns.distplot(df.loc[:,feature], norm_hist=True, ax = ax1)

    ax2 = fig.add_subplot(grid[1, :2])
    ax2.set_title('QQ_plot')
    stats.probplot(df.loc[:,feature], plot = ax2)

    ax3 = fig.add_subplot(grid[:, 2])
    ax3.set_title('Box Plot')
    sns.boxplot(df.loc[:,feature], orient='v', ax = ax3 );

    print("Skewness: "+ str(train['SalePrice'].skew().round(3))) 
    print("Kurtosis: " + str(train['SalePrice'].kurt().round(3)))

#---> Analyzing the Target (what I want to predict)   ** SalePrice ** 
    
multi_plotting (train,'SalePrice')

Here we can noticed skewness (large tail - right - therefore we have to standardize)

In [ ]:
# Transforming 'Sale Price' using log1p
train["SalePrice"] = np.log1p(train["SalePrice"])
multi_plotting (train,'SalePrice')

### B. Dropping Columns
1. Filter columns , if they have more than 60 % Missing/Nan , They Will be Dropped 
2. I will work Parellaly Train and Data Set, I can work with Data but I have to define 'saleprice'.notnull
3. The Data Description Mentioned all NAN values can be interprate as NONE / Dont have 

In [ ]:
# --> Columns missing Rate

def missing_rate(frame):
    TL = frame.isna().sum()/len(frame)  # Total len
    TCMR = TL[TL >0.6].index  #Total % Missing Rate in Columns
    frame.drop(TCMR, axis= 1 , inplace = True)
    print(f'Colums to be dropped:{TCMR}')
    
# --> Checking trainand Data 
missing_rate(train)
missing_rate(data)

###  C. Understanding Variables 
1. The key Issue Here is to Choose The Best Variables, however the "0" and Nan Values make it hard, therefore we must fill them and them drop no significant variables.
2. Basically we have to distinguishe Between A nominal variable which has no intrinsic ordering to its categories. For example, gender is a categorical variable having two categories (male and female). An ordinal variable has a clear ordering.For example, temperature as a variable with three orderly categories (low, medium and high)
3. Categorical variable must be transformed into numerical value 1,2,3,4 etc , but It has to make sense *** I.e "PoolQC" Ex=4, Ta=2 etc, because the model will understand that 4 is greater than 3 and will affect the final output but "MiscFeature" has tennis Court, Garage ,Elevator ,these tags are not related so The best approach will be OnehotEncoder ***
4. Fill columns with Numerical Values

In [ ]:
# --> General Fuctions 

def corr_heat (frame):   #<---Heat Map
    correlation = frame.corr()
    f,ax = plt.subplots(figsize=(30,20))
    mask = np.triu(correlation)
    sns.heatmap(correlation, annot=True, mask=mask,ax=ax,cmap='viridis')
    bottom,top = ax.get_ylim()
    ax.set_ylim(bottom+ 0.5, top - 0.5)
    
def box_plot (column1,column2,column3,data):  #<-- Box_plot
    f,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,5))
    sns.boxplot(y='SalePrice', x=column1 , data=data,ax=ax1)
    sns.boxplot(y='SalePrice', x=column2 , data=data,ax=ax2)
    sns.boxplot(y='SalePrice', x=column3 , data=data,ax=ax3)
    
def finding_zeros (frame): #<--- Finding 0 Values
    Zeros = frame[frame==0].count().sort_values(ascending=False)
    print(Zeros,"/" ,end ='')

### 1. Numeric & Float  *** ---> Into the Battle Field ***

In [ ]:
# -- > Extracting the Columns Name 
numeric = data.dtypes[data.dtypes != 'object'].index
tnumeric = train.dtypes[train.dtypes != 'object'].index

# Creating sub_dataset  - all columsn typed integer / float
integer = data[numeric]
tinteger = train[tnumeric]

# Filling Missing Values 
integer.fillna(0, inplace=True)
tinteger.fillna(0 , inplace = True)

In [ ]:
corr_heat(tinteger)

***The heatmap shows many columns with strongl correlation, I.e Yearbuild/GarageBuild *** the target is to reduce the dimensionality (number of columns), let's Find the columns correlated and then sort by the Sale price correlation

In [ ]:
FC = tinteger.corr() # Features Correlation
Target = tinteger.corr()['SalePrice'].to_frame().reset_index() #Feature Correlation related to SalePrice
FR = FC.unstack().to_frame(name='Correlation') # Feature Relation
Feature = FR[(FR['Correlation']>=0.8)&(FR['Correlation']<1)].sort_values(by='Correlation', ascending = False).reset_index()
Feature.head(5)

In [ ]:
Final  = Feature.merge(Target,left_on='level_1', right_on='index')
Final

We will Drop the following Columns because they are strongly Related , but btween them I will chose the one with stronger relation with Saleprice, (iD will be dropped as well)

In [ ]:
tinteger.drop(['GarageArea','GarageYrBlt','TotRmsAbvGrd','Id'],axis=1, inplace=True)
integer.drop(['GarageArea','GarageYrBlt','TotRmsAbvGrd','Id'],axis=1, inplace=True)

### 1.1. Feature Engineer --  Dealing with 0 values 

In [ ]:
t = tinteger[tinteger==0].count().sort_values(ascending=False).head(25)
d = integer[integer==0].count().sort_values(ascending=False).head(25)
source = {'Train':t, 'Data':d}
Zeros = pd.DataFrame.from_dict(source)
Zeros.sort_values(by=['Train','Data'],ascending=False)

### 1.2. Dealing with ***'3SsnPorch','ScreenPorch','EnclosedPorch','OpenPorchSF'*** which is the Area outside the House 
1. 24 with 3SsnPorch, 116 ScreenPorch , 804 OpenPorchSf, i will sum all of them and then create a columns  Have/No have (0/1)

In [ ]:
#--> Train Dealing with Porch 

tinteger['Porch'] = tinteger['3SsnPorch']+tinteger['ScreenPorch']+tinteger['EnclosedPorch']+tinteger['OpenPorchSF']
tinteger['YNPorch'] = tinteger['Porch'].apply(lambda x : 1 if x>0 else 0)
tinteger.drop(['3SsnPorch','ScreenPorch','EnclosedPorch','OpenPorchSF','PoolArea'], axis= 1, inplace=True)

# --> Data

integer['Porch'] = integer['3SsnPorch']+integer['ScreenPorch']+integer['EnclosedPorch']+integer['OpenPorchSF']
integer['YNPorch'] = integer['Porch'].apply(lambda x : 1 if x>0 else 0)
integer.drop(['3SsnPorch','ScreenPorch','EnclosedPorch','OpenPorchSF','PoolArea'], axis= 1, inplace=True)


In [ ]:
tinteger

### 1.3. Dealing With **'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF' = TotalBsmtSF *** Please notice the dependency 
1. BsmtFinSF1 +BsmtFinSF2+ BsmtUnfSF = 'TotalBsmtSF' , now I will Create a column "Bsmntbuilt" , 


In [ ]:
tinteger[['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']].tail(5)

In [ ]:
def cat_basmentfin (df):

    # Creating New Column
    df['Bsmntbuilt'] = 0

    # --> 2 ="Built (type1/type2)"  1 = "no built but available" and 0 means No Basement 
    A= df[(df['BsmtFinSF1']>0) | (df['BsmtFinSF2']>0) & (df['BsmtUnfSF'] >= 0)].index
    B= df[(df['BsmtFinSF1']==0) & (df['BsmtFinSF2']==0) & (df['BsmtUnfSF'] > 0)].index
    C= df[(df['BsmtFinSF1']==0) & (df['BsmtFinSF2']==0) & (df['BsmtUnfSF'] == 0)].index

    # --> Replacing 
    df.loc[A, 'Bsmntbuilt'] = 2
    df.loc[B, 'Bsmntbuilt'] = 1
    df.loc[C, 'Bsmntbuilt'] = 0
    
    #Dropping
    df.drop(['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis= 1 , inplace=True)


In [ ]:
tinteger.iloc[:,10:]

In [ ]:
# Categorizing Basement built

cat_basmentfin(tinteger) #---> Train 
cat_basmentfin(integer) #---> data

### 1.4.  Dealing with With FullBath/HalfBath
1. FullBath.unique() = [0,1,2,3] and HalflBath.unique(0,1,2) , After iterating all the Combination this is the final Result

In [ ]:
def basmentbath (df):  
    
    No_BsmtFullBath=df[(df['BsmtFullBath']==0) & (df['BsmtHalfBath']==0)].index
    Only_halfBath=df[(df['BsmtFullBath']==0) & (df['BsmtHalfBath']==1)].index
    Two_halfBath =df[(df['BsmtFullBath']==0) & (df['BsmtHalfBath']==2)].index
    Only_FullBath=df[(df['BsmtFullBath']==1) & (df['BsmtHalfBath']==0)].index
    One_Full_Half = df[(df['BsmtFullBath']==1) & (df['BsmtHalfBath']==1)].index
    Two_Full = df[(df['BsmtFullBath']==2) & (df['BsmtHalfBath']==0)].index
    Three_Full = df[(df['BsmtFullBath']==3) & (df['BsmtHalfBath']==0)].index

    # --> 9. Creating new Columns and Replacing
    df['BsmtBathCat'] = 0

    #--> Replacing 6=3 FullBath , 5=2 FullBath, 4=1Full/1Half, 3= 1FullBath, 5=2 HalfBath , 1=1 Half, 7=No Bath in Basement 
    df.loc[No_BsmtFullBath, 'BsmtBathCat'] = 0
    df.loc[Only_halfBath, 'BsmtBathCat'] = 1
    df.loc[Two_halfBath, 'BsmtBathCat'] = 2
    df.loc[Only_FullBath, 'BsmtBathCat'] = 3
    df.loc[One_Full_Half, 'BsmtBathCat'] = 4
    df.loc[Two_Full, 'BsmtBathCat'] = 5
    df.loc[Three_Full, 'BsmtBathCat'] = 6
    
    #Dropping
    df.drop(['BsmtFullBath','BsmtHalfBath'],axis= 1 , inplace=True)

In [ ]:
# Categorizing Basement Bath

basmentbath(tinteger) #---> Train 
basmentbath(integer) #---> data

In [ ]:
tinteger.iloc[:,13:]

### 1.5.  Dealing with 2nd floor 
1. 1stFlrSf + 2ndFlrSf = 'GrLivArea' , then I will Create a Column "1st_2nd_floor , where 0/1 = Have/no Have (2nd floor)

In [ ]:
integer['2ndFloor']=integer['2ndFlrSF'].apply(lambda x: 0 if x==0 else 1)
tinteger['2ndFloor']=tinteger['2ndFlrSF'].apply(lambda x: 0 if x==0 else 1)

integer.drop(['1stFlrSF','2ndFlrSF'],axis =1, inplace = True)
tinteger.drop(['1stFlrSF','2ndFlrSF'],axis =1, inplace = True)

In [ ]:
# ---> train 
tinteger= tinteger.join(train['Neighborhood'])
lot_mean = tinteger.groupby("Neighborhood")["LotFrontage"].mean().round().to_dict()
tinteger.loc[(tinteger['LotFrontage']==0) & (tinteger['Neighborhood'].isin(lot_mean.keys())), 'LotFrontage'] = tinteger['Neighborhood'].map(lot_mean)
tinteger.drop('Neighborhood',axis=1 , inplace=True)

# ---> test
integer= integer.join(data['Neighborhood'])
lot_mean = integer.groupby("Neighborhood")["LotFrontage"].mean().round().to_dict()
integer.loc[(integer['LotFrontage']==0) & (integer['Neighborhood'].isin(lot_mean.keys())), 'LotFrontage'] = integer['Neighborhood'].map(lot_mean)
integer.drop('Neighborhood',axis=1 , inplace=True)

In [ ]:
tinteger.iloc[:,13:]

### 1.7. Dealing with LowQualFinSF,  MiscVal
1. LowQualFinSF: Low quality finished square feet (all floors) so if it is 0 is because has been built (so I iwll Drop it)
2. MiscVal: $Value of miscellaneous feature, Well there is no way that we can know that (drop)

In [ ]:
integer.drop(['LowQualFinSF','MiscVal'],axis =1, inplace = True)
tinteger.drop(['LowQualFinSF','MiscVal'],axis =1, inplace = True)

### 1.7. Final View

In [ ]:
t = tinteger[tinteger==0].count().sort_values(ascending=False).head(14)
d = integer[integer==0].count().sort_values(ascending=False).head(14)
source = {'Train':t, 'Data':d}
Zeros = pd.DataFrame.from_dict(source)
Zeros.sort_values(by=['Train','Data'],ascending=False)

In [ ]:
corr_heat(tinteger)

In [ ]:
box_plot('MoSold','YrSold','KitchenAbvGr',tinteger)

In [ ]:
box_plot('OverallCond','MSSubClass','KitchenAbvGr',tinteger)

We can Notice that there is no relation among this variables , instead seems to behave as a constant look at the Yr Sold/MoSold there is no significant changes so i will Drop ***'MoSold','YrSold','KitchenAbvGr'***

In [ ]:
integer.drop(['MoSold','YrSold','KitchenAbvGr'],axis =1, inplace = True)
tinteger.drop(['MoSold','YrSold','KitchenAbvGr'],axis =1, inplace = True)

### 2. Categorical Ordinal 
1. I will evaluated from 0-5 / 0-6 in some cases   
2. I created Dicctionaties to replace the values based on the Description
3. OnelabelEncoder can do this but it will give them random weight and that would affect the final model

In [ ]:
categorical = data[['KitchenQual','FireplaceQu','GarageQual','GarageFinish','GarageCond','BsmtCond','BsmtExposure','BsmtQual','BsmtFinType2',
                    'BsmtFinType1','ExterCond','ExterQual','HeatingQC','LandSlope','LotShape','PavedDrive','Street',
                    'BldgType','Functional','SaleType','GarageType','Electrical','Foundation','LandContour','RoofMatl','RoofStyle']]


tcategorical = train[['SalePrice','KitchenQual','FireplaceQu','GarageQual','GarageFinish','GarageCond','BsmtCond','BsmtExposure','BsmtQual','BsmtFinType2',
                     'BsmtFinType1','ExterCond','ExterQual','HeatingQC','LandSlope','LotShape','PavedDrive','Street',
                     'BldgType','Functional','SaleType','GarageType','Electrical','Foundation','LandContour','RoofMatl','RoofStyle']]


In [ ]:
# ---->> Creating Dicctionaty (I created with nltk)

kit ={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0}               #KitchenQual: Kitchen quality 
#alley = {"Grvl":1,"Pave":2,"None":0}                      #Alley: Type of alley access to property 
fire ={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0}       #FireplaceQu: Fireplace quality
garagequa ={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0}     #GarageQual: Garage quality 
garage_finish = {"Fin":3,"RFn":2,"Unf":1,"None":0}        #GarageFinish: Interior finish of the garage
garagecond= {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0} #GarageCond: Garage condition 
bsmtcond= {"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0}          #BsmtCond: Evaluates the general condition of the basement
bsmtexposure = {"Gd":4,"Av":3,"Mn":2,"No":1,"None":0}     #BsmtExposure: Refers to walkout or garden level walls
bsmtqual = {"Ex":4,"Gd":3,"TA":2,"Fa":1,"None":0}         #BsmtQual: Evaluates the height of the basement
bsmtype2={"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1,"None":0}   #BsmtFinType2: Rating of basement finished area (if multiple types)
bsmtype1 = {"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1,"None":0} #BsmtFinType1: Rating of basement finished area
#poolqc={"Ex":3,"Gd":2,"Fa":1,"None":0}                    #PoolQC: Pool quality
extercond = {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}         #ExterCond: Evaluates the present condition of the material on the exterior
exterqual =  {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}        #ExterQual: Evaluates the quality of the material on the exterior
#fence = {"GdPrv":4,"MnPrv":3,"GdWo":2,"MnWw":1, "None":0}   #Fence: Fence quality
heatingqc = {"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}         #HeatingQC: Heating quality and condition
landslope={"Gtl":3,"Mod":2,"Sev":1}                        #LandSlope: Slope of property
lotshape={"Reg":4,"IR1":3,"IR2":2,"IR3":1}                #LotShape: General shape of property
pavedrive= {"Y":3,"P":2,"N":1}                            #PavedDrive: Paved driveway
street={"Grvl":2,"Pave":1}                                #Street: Type of road access to property
bldgtype = {'1Fam': 1, '2fmCon': 2, 'Duplex': 3, 'TwnhsE': 4, 'Twnhs': 5}
functional = {'Typ': 8,'Min1': 7,'Min2': 6,'Mod': 5,'Maj1': 4,'Maj2': 3,'Sev': 2,'Sal': 1,'None':1}
saletype ={'WD': 10,'CWD': 9,'VWD': 8,'New': 7,'COD': 6,'Con': 5,'ConLw': 4,'ConLI': 3,'ConLD': 2,'Oth': 1,'None':0}
garage={'2Types': 6, 'Attchd': 5, 'Basment': 4, 'BuiltIn': 3, 'CarPort': 2, 'Detchd': 1, 'None': 0}
electrical={'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix': 1,'None':0}
foundation={'BrkTil': 6, 'CBlock': 5, 'PConc': 4, 'Slab': 3, 'Stone': 2, 'Wood': 1}
landcontour={'Lvl': 4, 'Bnk': 3, 'HLS': 2, 'Low': 1}
roofmatl={'ClyTile': 8, 'CompShg': 7, 'Membran': 6, 'Metal': 5, 'Roll': 4, 'Tar&Grv': 3, 'WdShake': 2, 'WdShngl': 1}
roofstyle ={'Flat': 6, 'Gable': 5, 'Gambrel': 4, 'Hip': 3, 'Mansard': 2, 'Shed': 1}

In [ ]:
# ---> Replacing Values General fuction
for i in tcategorical:  #-->Train
    tcategorical[i].fillna('None',inplace=True)
    
for i in categorical: #--> Test
    categorical[i].fillna('None',inplace=True)

In [ ]:
# --> a quick glance 

for i in tcategorical.columns[1:].to_list():  #---> Train
    v= tcategorical[i].unique()
    print(i, v)

In [ ]:
# -->  Joining Dict 

list_dict =[kit,fire,garagequa,garage_finish,garagecond,bsmtcond,bsmtexposure,bsmtqual,bsmtype2,
            bsmtype1,extercond,exterqual,heatingqc,landslope,lotshape,pavedrive,street,
            bldgtype,functional,saletype,garage,electrical,foundation,landcontour,roofmatl,roofstyle]
    
#-->  Replacing using Loop + Fuction + Dicct  

def replace_t (i,diccionaty): #fuction for Train 
    tcategorical[i].replace(diccionaty, inplace=True) 
    
def replace (i,diccionaty): #fuction for data 
    categorical[i].replace(diccionaty, inplace=True) 

# --->  Replacing / Mapping values   
    
for i,j in zip(tcategorical.columns[1:].to_list() ,list_dict):  #[1:] because Im not inclueding SalePrice
    replace_t(i,j)
    
for i,j in zip(categorical.columns.to_list() ,list_dict):  #[1:] because Im not inclueding SalePrice
    replace(i,j)
    
for i in categorical:
    v= categorical[i].unique()
    print(i, v)

In [ ]:
corr_heat(tcategorical)

 Before going foward I will  try to reduce the dimensinalty by dropping those columns that has many 0 / None

In [ ]:
for i in tcategorical:
    v = tcategorical[i].value_counts().to_frame().reset_index().iloc[0].to_list()
    w= tcategorical[i].value_counts().reset_index().iloc[0][0]
    if w == 0:
        print(i,v)

In [ ]:
tcategorical.drop('FireplaceQu',axis=1, inplace=True)
categorical.drop('FireplaceQu',axis=1, inplace=True)

### 3. Categorical Nominal 
1. From this point I will work with Nominal (data)

In [ ]:
nominal =data[['Utilities','Exterior1st','Exterior2nd','MasVnrType',
                     'MSZoning','CentralAir','Condition1','Condition2','Heating','HouseStyle','LotConfig',
                     'Neighborhood','SaleCondition']]


#--> Finding Null Values 
nominal.isnull().sum().head(6)

In [ ]:
# -- using Label Encoder 

for i in nominal:
    encoding = pd.get_dummies(nominal[i], prefix=i, drop_first= True)
    nominal= pd.concat([nominal,encoding],axis=1)
    nominal.drop(i,axis=1, inplace=True)

nominal

In [ ]:
#--> Removing Feature with low Variance

all_features = nominal.keys()
# Removing features.
nominal = nominal.drop(nominal.loc[:,(nominal==0).sum()>=(nominal.shape[0]*0.9994)],axis=1)
nominal = nominal.drop(nominal.loc[:,(nominal==1).sum()>=(nominal.shape[0]*0.9994)],axis=1) 
# Getting and printing the remaining features.
remain_features = nominal.keys()
remov_features = [st for st in all_features if st not in remain_features]
print(len(remov_features), 'features were removed:', remov_features)

# 4. Combining Data Sets

In [ ]:
print(integer.shape, categorical.shape, nominal.shape)

In [ ]:
new_data = pd.concat([integer,categorical,nominal], axis = 1 , sort=False)
new_data

In [ ]:
train_data =new_data.iloc[:1460].reset_index(drop=True)
test_data = new_data.iloc[1460:].reset_index(drop=True)

# 5. Machine Learning Section
1. It is important to use np.expm1 at the predicttion because of the transformation done in the very beggining 

## 5.1. Linear Regression 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split

X = train_data
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

LR = LinearRegression().fit(X_train,y_train)
y_pred_LR = LR.predict(test_data)


test_Id=test['Id']
my_submission = pd.DataFrame({'Id': test_Id, 'SalePrice': np.expm1(y_pred_LR)})
my_submission.to_csv('KaggelsubmissionLR.csv', index=False)

with Linear Regression I got 0.13 error rate in Kaggel. 

## 5.2. XgBoost.Regressor  

In [ ]:
from sklearn.model_selection import train_test_split


train_data = train_data.astype(int)
test_data = test_data.astype(int)

X = train_data
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
import xgboost 
classifier = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=2, min_child_weight=4, missing=None, n_estimators=900,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=0)

classifier = classifier.fit(X_train,y_train)
y_pred_XG = classifier.predict(test_data)
sub = pd.DataFrame()
sub = pd.DataFrame()
sub['Id'] = test['Id']
sub['SalePrice'] = np.expm1(y_pred_XG)
sub.to_csv('KaggelXBoostsubmission.csv',index=False)

with Linear Regression I got 0.12 error rate in Kaggel. 

## 5.3. Other
1. Im still working on that 